In [1]:
# -
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row, gridplot
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

from scipy.io import loadmat
from scipy.signal import hilbert, welch


from scipy.signal import butter, lfilter
from fakespikes.util import create_times
from collections import defaultdict

# -
import os
import h5py
import numpy as np

from tqdm import tqdm
from glob import glob
from pprint import pprint

from scipy.io import loadmat

from fakespikes.util import create_times
from fakespikes.util import create_psd
from alphalogical.util import find_bursts
from alphalogical.util import find_bursts2
from alphalogical.util import find_notbursts
from alphalogical.util import butter_bandpass_filter
from alphalogical.util import load_smith_foof_results
from alphalogical.util import load_smith_rates
from alphalogical.util import load_smith_lfps
from alphalogical.util import find_smith_bursts
from alphalogical.util import analyze_smith_bursts

Loading BokehJS ...

# Pathing

In [2]:
DATA_PATH = "/Users/type/Data/Smith/data"


# Load data and segment files

In [3]:
good_files = ["Bo130408_s6ae_fixblank_active_0001_converted_ns2.mat",
"Bo130408_s6ae_fixblank_active_0002_converted_ns2.mat",
"Bo130409_s7ae_fixblank_active_0003_converted_ns2.mat",
"Wi130116_s51ae_fixblank_active_0001_converted_ns2.mat",
"Bo130404_s4ae_fixblank_active_0002_converted_ns2.mat",
"Bo130405_s5ae_fixblank_active_0001_converted_ns2.mat",
"Bo130405_s5ae_fixblank_active_0002_converted_ns2.mat",
"Bo130405_s5ae_fixblank_active_0003_converted_ns2.mat",
"Bo130418_s12ae_fixblank_active_0002_converted_ns2.mat",
"Wi121219_s43ae_fixblank_active_0001_converted_ns2.mat",
"Wi121219_s43ae_fixblank_active_0002_converted_ns2.mat",
"Wi130129_s55ae_fixblank_active_0001_converted_ns2.mat",
"Wi130129_s55ae_fixblank_active_0002_converted_ns2.mat",
"Wi130205_s58ae_fixblank_active_0001_converted_ns2.mat",
"Wi130205_s58ae_fixblank_active_0002_converted_ns2.mat",
"Wi130207_s59ae_fixblank_active_0001_converted_ns2.mat",
"Wi130207_s59ae_fixblank_active_0002_converted_ns2.mat",
"Wi130207_s59ae_fixblank_active_0003_converted_ns2.mat",
"Wi130208_s60ae_fixblank_active_0001_converted_ns2.mat",
"Wi130211_s61ae_fixblank_active_0001_converted_ns2.mat",
"Wi130212_s62ae_fixblank_active_0001_converted_ns2.mat"]

pprint([(i, fi) for i, fi in enumerate(good_files)])

[(0, 'Bo130408_s6ae_fixblank_active_0001_converted_ns2.mat'),
 (1, 'Bo130408_s6ae_fixblank_active_0002_converted_ns2.mat'),
 (2, 'Bo130409_s7ae_fixblank_active_0003_converted_ns2.mat'),
 (3, 'Wi130116_s51ae_fixblank_active_0001_converted_ns2.mat'),
 (4, 'Bo130404_s4ae_fixblank_active_0002_converted_ns2.mat'),
 (5, 'Bo130405_s5ae_fixblank_active_0001_converted_ns2.mat'),
 (6, 'Bo130405_s5ae_fixblank_active_0002_converted_ns2.mat'),
 (7, 'Bo130405_s5ae_fixblank_active_0003_converted_ns2.mat'),
 (8, 'Bo130418_s12ae_fixblank_active_0002_converted_ns2.mat'),
 (9, 'Wi121219_s43ae_fixblank_active_0001_converted_ns2.mat'),
 (10, 'Wi121219_s43ae_fixblank_active_0002_converted_ns2.mat'),
 (11, 'Wi130129_s55ae_fixblank_active_0001_converted_ns2.mat'),
 (12, 'Wi130129_s55ae_fixblank_active_0002_converted_ns2.mat'),
 (13, 'Wi130205_s58ae_fixblank_active_0001_converted_ns2.mat'),
 (14, 'Wi130205_s58ae_fixblank_active_0002_converted_ns2.mat'),
 (15, 'Wi130207_s59ae_fixblank_active_0001_converted_ns2.

In [4]:
# Generate segment file names based on the good list
rate_files = []
for fi in good_files:
    fi_name = os.path.splitext(fi)[0]
    fi_name = fi_name.split("_")[:5]
    fi_name = "_".join(fi_name)

    new_name = "{}.mat".format(fi_name)

    rate_files.append(new_name)

pprint([(i, fi) for i, fi in enumerate(rate_files)])

[(0, 'Bo130408_s6ae_fixblank_active_0001.mat'),
 (1, 'Bo130408_s6ae_fixblank_active_0002.mat'),
 (2, 'Bo130409_s7ae_fixblank_active_0003.mat'),
 (3, 'Wi130116_s51ae_fixblank_active_0001.mat'),
 (4, 'Bo130404_s4ae_fixblank_active_0002.mat'),
 (5, 'Bo130405_s5ae_fixblank_active_0001.mat'),
 (6, 'Bo130405_s5ae_fixblank_active_0002.mat'),
 (7, 'Bo130405_s5ae_fixblank_active_0003.mat'),
 (8, 'Bo130418_s12ae_fixblank_active_0002.mat'),
 (9, 'Wi121219_s43ae_fixblank_active_0001.mat'),
 (10, 'Wi121219_s43ae_fixblank_active_0002.mat'),
 (11, 'Wi130129_s55ae_fixblank_active_0001.mat'),
 (12, 'Wi130129_s55ae_fixblank_active_0002.mat'),
 (13, 'Wi130205_s58ae_fixblank_active_0001.mat'),
 (14, 'Wi130205_s58ae_fixblank_active_0002.mat'),
 (15, 'Wi130207_s59ae_fixblank_active_0001.mat'),
 (16, 'Wi130207_s59ae_fixblank_active_0002.mat'),
 (17, 'Wi130207_s59ae_fixblank_active_0003.mat'),
 (18, 'Wi130208_s60ae_fixblank_active_0001.mat'),
 (19, 'Wi130211_s61ae_fixblank_active_0001.mat'),
 (20, 'Wi130212_s

In [5]:
# Generate rate file names based on the good list
seg_files = []
for fi in good_files:
    fi_name = os.path.splitext(fi)[0]
    new_name = "{}_segments.csv".format(fi_name)

    seg_files.append(new_name)

pprint([(i, fi) for i, fi in enumerate(seg_files)])

[(0, 'Bo130408_s6ae_fixblank_active_0001_converted_ns2_segments.csv'),
 (1, 'Bo130408_s6ae_fixblank_active_0002_converted_ns2_segments.csv'),
 (2, 'Bo130409_s7ae_fixblank_active_0003_converted_ns2_segments.csv'),
 (3, 'Wi130116_s51ae_fixblank_active_0001_converted_ns2_segments.csv'),
 (4, 'Bo130404_s4ae_fixblank_active_0002_converted_ns2_segments.csv'),
 (5, 'Bo130405_s5ae_fixblank_active_0001_converted_ns2_segments.csv'),
 (6, 'Bo130405_s5ae_fixblank_active_0002_converted_ns2_segments.csv'),
 (7, 'Bo130405_s5ae_fixblank_active_0003_converted_ns2_segments.csv'),
 (8, 'Bo130418_s12ae_fixblank_active_0002_converted_ns2_segments.csv'),
 (9, 'Wi121219_s43ae_fixblank_active_0001_converted_ns2_segments.csv'),
 (10, 'Wi121219_s43ae_fixblank_active_0002_converted_ns2_segments.csv'),
 (11, 'Wi130129_s55ae_fixblank_active_0001_converted_ns2_segments.csv'),
 (12, 'Wi130129_s55ae_fixblank_active_0002_converted_ns2_segments.csv'),
 (13, 'Wi130205_s58ae_fixblank_active_0001_converted_ns2_segments.cs

# Pick the dataset to analyze 

Name it `x`.

In [6]:
n = 0

In [7]:
# Load all its segments
header, segments = load_smith_foof_results(
    os.path.join(DATA_PATH, seg_files[n]))

print("Header {}".format(header))
print("Segment N: {}".format(len(segments)))
print("Segment row {}".format(segments[0]))

Header ['m', 'c', 'i', 'j', 'center', 'power', 'bw']
Segment N: 11805
Segment row (0, 49, 1131712, 1141712, 13.0381994977, 0.3680282355, 2.83154123279)


# Pick a segment

In [8]:
m = 2004

In [9]:
# Get indices
m, c, i, j, center, power, bw = segments[m]
print(i, j)

(3163175, 3173175)


# Load data for channel `c`

In [10]:
# Get the LFP
fs, times, x_ref = load_smith_lfps(
    os.path.join(DATA_PATH, good_files[n]), 
    c)

# Parse time
T = times.max()
times_m = times[i:j]

# For burst thresholding we use whole electrode
x_lfp = x_ref[i:j]

print("T: {}".format(T))
print("ref shape: {}".format(x_ref.shape))
print("lfp_m shape: {}".format(x_lfp.shape))

T: 3486.306
ref shape: (3486306,)
lfp_m shape: (10000,)


In [11]:
# get the rate
_, x_rate_ref = load_smith_rates(
    os.path.join(DATA_PATH, rate_files[n]), 
    c, (0, T), 1/fs)

x_rate = x_rate_ref[i:j]

print("ref shape: {}".format(x_rate_ref.shape))
print("Rate shape: {}".format(x_rate.shape))

ref shape: (3486305,)
Rate shape: (10000,)


## Explore raw timeseries

In [12]:
# Intial window (s)
t = float(x_lfp.size / fs)
win = (0, t)

p = figure(plot_width=600, plot_height=150)
p.line(times_m, x_lfp, color='black', line_width=1)
p.xaxis.axis_label = "Time (s)"
p.yaxis.axis_label = "LFP (mV)"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

freqs, psd = welch(x_lfp, fs=fs, window='hanning', nperseg=fs)

p = figure(plot_width=250, plot_height=150)
p.line(freqs, psd, color='black', alpha=0.9, line_width=2);
p.x_range = Range1d(0, 50)
p.xaxis.axis_label = "Freq (Hz)"
p.yaxis.axis_label = "Power (AU)"
show(p)

In [13]:
p = figure(plot_width=600, plot_height=150)
p.line(times_m, x_rate, color='black')
p.xaxis.axis_label = "Time (s)"
p.yaxis.axis_label = "Rate (Hz)"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Find the bursts in m

In [14]:
bursts, alpha, alpha_pow, M = find_smith_bursts(x_ref, x_lfp, center, fs)

# Extract HG power as well
hg = butter_bandpass_filter(x_lfp, 100, 150, fs, order=2)
hg_pow = np.abs(hilbert(hg))

# Plot the threshold and the bursts

### LFP

In [15]:
x_filt = butter_bandpass_filter(x_lfp, 3, 100, fs, order=2)

p = figure(plot_width=800, plot_height=250)
p.line(times_m, x_filt, color='black', alpha=0.3)
p.line(times_m, alpha, color='red', alpha=0.6)
p.line(times_m, alpha_pow, color='red', line_width=5, alpha=0.2)
p.line(times_m, M, color='gold', line_dash="dashed", line_width=5)
for b in bursts:
    start, stop = min(b), max(b)
    p.line(x=times_m[start:stop], y=x_filt.max(), color="blue", line_width=5, alpha=0.2)
p.xaxis.axis_label = "Time (s)"
p.yaxis.axis_label = "Power (AU)"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

### Rate

In [16]:
p = figure(plot_width=800, plot_height=250)
p.line(times_m, x_rate, color='black', alpha=0.3)
for b in bursts:
    start, stop = min(b), max(b)
    p.line(x=times_m[start:stop], y=x_rate.max(), color="blue", line_width=5, alpha=0.2)
p.xaxis.axis_label = "Time (s)"
p.yaxis.axis_label = "Power (AU)"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Analyze bursts

In [18]:
results = analyze_smith_bursts(m, bursts, alpha_pow, hg_pow, x_rate, fs)
pprint(results)

defaultdict(<type 'list'>, {'burst_index': [0, 1, 2, 3], 'hg_power': [4.513132793077821, 5.2258154309260485, 5.009252089504675, 4.270225790824943], 'rate': [0.0, 0.0, 0.0, 0.0], 'n_bursts': [4, 4, 4, 4], 'results_code': [2004, 2004, 2004, 2004], 'burst_time': [0.061, 0.186, 0.158, 0.39], 'alpha_power': [73.83006729530308, 98.2832164294815, 99.79132038774836, 97.70518524224372]})


# Analyze all segments for a files

- TODO

In [ ]:
# TODO

# Plot distributions of burst statistics

- TODO adapt below

### Burst distributions

In [ ]:
hist, edges = np.histogram(bursts["L"], density=False, bins=100)
p = figure(plot_width=300, plot_height=200)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], color="black")
p.xaxis.axis_label = "L (s)"
p.yaxis.axis_label = "Count"
show(p)

### Alpha

In [ ]:
hist, edges = np.histogram(bursts["alpha"], density=False, bins=300)
p = figure(plot_width=300, plot_height=200)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], color="black")
p.xaxis.axis_label = "Alpha power"
p.yaxis.axis_label = "Count"
show(p)

In [ ]:
hist1, edges1 = np.histogram(bursts["alpha_burst"], density=True, bins=50)
hist2, edges2 = np.histogram(bursts["alpha_osc"], density=True, bins=50)

p = figure(plot_width=300, plot_height=200)
p.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:], color="blue", legend="Burst", alpha=0.3)
p.quad(top=hist2, bottom=0, left=edges2[:-1], right=edges2[1:], color="purple", legend="Osc.", alpha=0.3)
p.xaxis.axis_label = "Alpha power"
p.yaxis.axis_label = "Count"
show(p)

### Gamma

In [ ]:
hist, edges = np.histogram(bursts["hg"], density=False, bins=300)
p = figure(plot_width=300, plot_height=200)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], color="black")
p.xaxis.axis_label = "Gamma power"
p.yaxis.axis_label = "Count"
show(p)

In [ ]:
hist1, edges1 = np.histogram(bursts["hg_burst"], density=True, bins=50)
hist2, edges2 = np.histogram(bursts["hg_osc"], density=True, bins=50)

p = figure(plot_width=300, plot_height=200)
p.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:], color="blue", legend="Burst", alpha=0.3)
p.quad(top=hist2, bottom=0, left=edges2[:-1], right=edges2[1:], color="purple", legend="Osc.", alpha=0.3)
p.xaxis.axis_label = "Gamma power"
p.yaxis.axis_label = "Count"
show(p)

### Alpha by HG power

In [ ]:
p = figure(plot_width=300, plot_height=200)
p.circle(bursts["hg"], bursts["alpha"], color='black', alpha=0.3)
p.xaxis.axis_label = "Gamma power"
p.yaxis.axis_label = "Alpha power"
show(p)

In [ ]:
p = figure(plot_width=300, plot_height=200)
p.circle(bursts["hg_burst"], bursts["alpha_burst"], color='blue', alpha=0.3, legend="Burst")
p.circle(bursts["hg_osc"], bursts["alpha_osc"], color='purple', alpha=0.3, legend="Osc.")
p.xaxis.axis_label = "Gamma power"
p.yaxis.axis_label = "Alpha power"
show(p)

### L by power

In [ ]:
p = figure(plot_width=300, plot_height=200)
p.circle(burst_lens, burst_hgpower, color='black', alpha=0.3)
p.xaxis.axis_label = "Length (s)"
p.yaxis.axis_label = "Gamma power"
show(p)

In [ ]:
p = figure(plot_width=300, plot_height=200)
p.circle(burst_lens, burst_alphapower, color='black')
p.xaxis.axis_label = "Length (s)"
p.yaxis.axis_label = "Alphs power"
show(p)

In [ ]:
p = figure(plot_width=300, plot_height=200)
p.circle(burst_lens[burst_mask], burst_alphapower[burst_mask], color='blue', alpha=0.3, legend="Burst")
p.circle(burst_lens[osc_mask], burst_alphapower[osc_mask], color='purple', alpha=0.3, legend="Osc")
p.xaxis.axis_label = "Length (s)"
p.yaxis.axis_label = "Alphs power"
show(p)